DarkNet 저장소를 복사한 후 Colab에서 쓸 수 있게 빌드합니다.

In [ ]:
!git clone https://github.com/jetsonai/darknet.git

%cd darknet 
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

!make -j$nproc

COCO 데이터셋으로 훈련한 YOLOv4-tiny를 이용한 추론을 수행하기 위해 가중치를 내려받습니다.

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights

In [12]:
# Colab에서 OpenCV 이미지를 보기 위한 패키지를 불러옵니다.
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [6]:
# darknet.py를 불러오고 이미지를 추론하기 위한 함수를 준비합니다.
import darknet

def image_detection(image_path, network, class_names, class_colors, thresh):
    # Darknet doesn't accept numpy images.
    # Create one with image we reuse for each detect
    width = darknet.network_width(network)
    height = darknet.network_height(network)
    darknet_image = darknet.make_image(width, height, 3)

    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_resized = cv2.resize(image_rgb, (width, height),
                               interpolation=cv2.INTER_LINEAR)

    darknet.copy_image_from_bytes(darknet_image, image_resized.tobytes())
    detections = darknet.detect_image(network, class_names, darknet_image, thresh=thresh)
    darknet.free_image(darknet_image)
    image = darknet.draw_boxes(detections, image_resized, class_colors)
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB), detections


In [10]:
# COCO dataset으로 훈련된 yolov4-tiny 신경망을 준비합니다.
config_file = "/content/darknet/cfg/yolov4-tiny.cfg"
data_file = "/content/darknet/cfg/coco.data"
weights = "/content/darknet/yolov4-tiny.weights"
threshold = 0.25

network, class_names, class_colors = darknet.load_network(
        config_file,
        data_file,
        weights,
        batch_size=1
    )

In [ ]:
# 이미지 추론을 수행합니다. result는 박스가 그려진 이미지이고, detections에는 객체 정보가 들어있습니다.
image_path = "/content/darknet/data/dog.jpg"
result, detections = image_detection(image_path, network, class_names, class_colors, threshold)
cv2_imshow(result)
print(detections)